In [ ]:
import mysql.connector
from mysql.connector import Error
import pandas as pd
import json
import time
from datetime import date
from datetime import datetime
import time

def create_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            passwd=user_password,
            database=db_name
        )
        print("Connection to MySQL DB successful")
    except Error as e:
        print(f"The error '{e}' occurred")

    return connection

In [1]:
mydb = create_connection("10.10.10.250", "dzhalilovd", "ROhaproD", "CCM_BILLING")

NameError: name 'create_connection' is not defined

In [ ]:
#query_yyyyy = '''select DST_NUMBER_IN as 'Проект', DATE(SETUP_TIME) as '.Дата', CONNECT_TIME, DISCONNECT_TIME, TIME(SETUP_TIME) as '.Время',
#           SRC_NUMBER_OUT as 'Номер А', DST_NUMBER_OUT as 'Номер Б', DST_NAME as 'Контрагент',
#           ROUND(TIMESTAMPDIFF(SECOND, CONNECT_TIME, DISCONNECT_TIME)/60, 2) as 'Длительность' 
#           from CCM_BILLING.CDR_RAW as r WHERE r.SETUP_TIME between '2021-01-01 00:00:00' and '2021-01-31 23:59:59'
#           AND (DST_NUMBER_IN LIKE '1201#%' OR DST_NUMBER_IN LIKE '2200#%') AND (DST_NAME LIKE 'GW_DIVA_M9' OR DST_NAME LIKE 'GW_SPACETEL%') AND CONNECT_TIME > 0'''


query_yyyytt = '''select DST_NUMBER_IN as 'Проект', DATE(SETUP_TIME) as '.Дата', CONNECT_TIME, DISCONNECT_TIME, TIME(SETUP_TIME) as '.Время',
           SRC_NUMBER_OUT as 'Номер А', DST_NUMBER_OUT as 'Номер Б', replace(DST_NAME, 'GW_SPACETEL_1', 'GW_SPACETEL*') as 'Контрагент',
           ROUND(TIMESTAMPDIFF(SECOND, CONNECT_TIME, DISCONNECT_TIME)/60, 2) as 'Длительность' 
           from CCM_BILLING.CDR_RAW as r WHERE r.SETUP_TIME between '2021-01-01 00:00:00' and '2021-01-31 23:59:59' and
           DST_NAME LIKE 'GW_SPACETEL%' AND CONNECT_TIME > 0'''



#query_test_yyyyy = '''select *
#               from CCM_BILLING.CDR_RAW as r WHERE r.SETUP_TIME between '2021-01-01 00:00:00' and '2021-01-31 23:59:59'
#               AND DST_NUMBER_IN LIKE '1201#%' AND DST_NUMBER_IN LIKE '2200#%' AND CONNECT_TIME > 0 '''


yyyyy = pd.read_sql_query(query_yyyytt, con = mydb)

In [ ]:
yyyyy.to_csv('GW_SPACETEL январь 2021.csv', sep=';', encoding='utf-8', index = False)

In [63]:
import pandas as pd
def main():
    with open ('Свод по тарифам(18.01.2021) — копия.txt', 'r', encoding='UTF8') as tariff, open ('GW_SPACETEL январь 2021.csv', 'r', encoding='UTF-8') as CDR, open ('Свод по тарифам(18.01.2021) — DEF.txt', 'r', encoding='UTF-8') as DEF:
        next(CDR)
        next(tariff)
        next(DEF)
        def CDR_tariff_func(CDR, tariff, DEF):
            dict_1 = {}
            dict_2 = {}
            dict_4 = {}
            dict_DEF = {}
            count1 = 0
            list_numb_CDR = []
            rr = 0
            for line_1 in CDR:
                if len(line_1.strip().split(';')[6]) == 11 and line_1.strip().split(';')[6][0:2] != '77' and (line_1.strip().split(';')[6][0] == '7' or line_1.strip().split(';')[6][0] == '8'):
                    list_join = []
                    numb_CDR = list(line_1.replace('\"', '').strip().split(';')[6])
                    numb_CDR[0:1] = ''
                    numb_CDR = ''.join(numb_CDR)
                    list_join.append(numb_CDR)
                    list_join.append('РФ')
                    numb_CDR1 = [numb_CDR] + ['РФ']
                else:
                    numb_CDR = line_1.replace('\"', '').strip().split(';')[6]
                    numb_CDR1 = [numb_CDR] + ['не РФ']
                list_numb_CDR.append(numb_CDR)
                count_numb_CDR = list_numb_CDR.count(numb_CDR)
                dict_1[tuple(map(str, [*numb_CDR1, line_1.replace('\"', '').strip().split(';')[7], count_numb_CDR]))] = line_1.replace('\"', '').strip().split(';')[0], line_1.replace('\"', '').strip().split(';')[1], line_1.replace('\"', '').strip().split(';')[2], line_1.replace('\"', '').strip().split(';')[3], line_1.replace('\"', '').strip().split(';')[4], line_1.replace('\"', '').strip().split(';')[5], line_1.replace('\"', '').strip().split(';')[8]

            for line_2 in tariff:
                dict_2[tuple(map(str, [line_2.strip().split(';')[0], line_2.strip().split(';')[5], line_2.strip().split(';')[10]]))] = line_2.strip().split(';')[2], line_2.strip().split(';')[3], line_2.strip().split(';')[4], line_2.strip().split(';')[5], line_2.strip().split(';')[6], line_2.strip().split(';')[7], line_2.strip().split(';')[8], line_2.strip().split(';')[9]
                
            for line_4 in DEF:
                dict_DEF[tuple(map(str, [line_4.strip().split(';')[0], line_4.strip().split(';')[1], line_4.strip().split(';')[5]]))] = line_4.strip().split(';')[2], line_4.strip().split(';')[3], line_4.strip().split(';')[4], line_4.strip().split(';')[6], line_4.strip().split(';')[7], line_4.strip().split(';')[8], line_4.strip().split(';')[9]
            return {'dict_1':dict_1, 'dict_2':dict_2, 'dict_DEF':dict_DEF}
        CDR_tariff_func1 = CDR_tariff_func(CDR, tariff, DEF)
        def list_final(CDR_tariff_func1):
            result = {}
            for numb_key, numb_val in CDR_tariff_func1['dict_1'].items():
                ggg = []
                for index_numb_key in range(len(numb_key[0]), 0, -1):
                    t = list(numb_key[0][0:index_numb_key].split())
                    t.insert(1, numb_key[2])  
                    t.insert(2, numb_key[1])
                    t = tuple(t)
                    if len(ggg)==0 and t in tuple(CDR_tariff_func1['dict_2'].keys()): 
                        ggg.append(index_numb_key)
                        result.setdefault('Проект',[]).append(numb_val[0])
                        result.setdefault('Дата',[]).append(numb_val[1])
                        result.setdefault('Время',[]).append(numb_val[4])
                        result.setdefault('Номер А',[]).append(numb_val[5])           
                        result.setdefault('Номер Б',[]).append(numb_key[0])
                        result.setdefault('Контрагент',[]).append(numb_key[1])
                        result.setdefault('Длительность',[]).append(numb_val[6])
                        result.setdefault('Тариф',[]).append(CDR_tariff_func1['dict_2'][t][2].replace(',', '.'))
                        result.setdefault('Сумма',[]).append(str(round(float(CDR_tariff_func1['dict_2'][t][2].replace(',', '.'))*float(numb_val[6]), 2)).replace(',', '.'))
                    else:
                        for numb_key_DEF, numb_val_DEF in CDR_tariff_func1['dict_DEF'].items():
                            if (int(numb_key_DEF[0]) < int(t[0]) < int(numb_key_DEF[1])) and t[1] == numb_key_DEF[2]:
                                ggg.append(index_numb_key)
                                t = (numb_key_DEF[0] +','+ numb_key_DEF[1] + ',' + numb_key_DEF[2]).split(',')
                                t = tuple(t)
                                result.setdefault('Проект',[]).append(numb_val[0])
                                result.setdefault('Дата',[]).append(numb_val[1])
                                result.setdefault('Время',[]).append(numb_val[4])
                                result.setdefault('Номер А',[]).append(numb_val[5])           
                                result.setdefault('Номер Б',[]).append(numb_key[0])
                                result.setdefault('Контрагент',[]).append(numb_key[1])
                                result.setdefault('Длительность',[]).append(numb_val[6])  
                                result.setdefault('Тариф',[]).append(CDR_tariff_func1['dict_DEF'][t][2].replace(',', '.'))
                                result.setdefault('Сумма',[]).append(round(float(CDR_tariff_func1['dict_DEF'][t][2].replace(',', '.'))*float(numb_val[6]), 2))
            return result
        fff = list_final(CDR_tariff_func1)
        def to_frame(fff):  
            df0 = pd.Series([],dtype=pd.StringDtype())
            df0 = pd.Series(fff['Проект'], name='Проект')
            df1 = pd.Series(fff['Дата'], name='Дата')
            df2 = pd.Series(fff['Время'], name='Время')
            df3 = pd.Series(fff['Номер А'], name='Номер А')
            df4 = pd.Series(fff['Номер Б'], name='Номер Б')
            df5 = pd.Series(fff['Длительность'], name='Длительность')
            df6 = pd.Series(fff['Контрагент'], name='Контрагент')
            df7 = pd.Series(fff['Тариф'], name='Тариф')
            df8 = pd.Series(fff['Сумма'], name='Сумма')
#            df9 = pd.Series(fff['Направление'], name='Напраление')
#            df10 = pd.Series(fff['Длительность из детализации'], name='Длительность из детализации')
#            df11 = pd.Series(fff['Тариф из детализации'], name='Тариф из детализации')
#            df12 = pd.Series(fff['Сумма из детализации'], name='Сумма из детализации')
#            df13 = pd.Series(fff['Номер Б из детализации'], name='Номер Б из детализации')
            df = pd.concat([df0,df1,df2,df3,df4,df5,df6,df7,df8], axis=1)
            df.to_excel('Billing_detail.xlsx', 'Sheet1', index=False)
        to_frame(fff)
main()